In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [ ]:
# inisialisasi jangka waktu
start_date = dt.datetime(year=2020, month=1, day=1)
end_date = dt.datetime(year=2022, month=1, day=1)

# akses data harga saham dari yahoo finance
test = data.DataReader(["PWON.JK", "ASRI.JK"], "yahoo", start_date, end_date)
test = pd.DataFrame(test)
test.head()

In [ ]:
# ambil kolom harga close
test = test["Close"]
test

In [ ]:
# Log of percentage change
pwon = test["PWON.JK"].pct_change().apply(lambda x: np.log(1+x))
pwon.head

In [ ]:
# variance
var_pwon = pwon.var()
var_pwon

In [ ]:
# Log of percentage change ASRI
asri = test["ASRI.JK"].pct_change().apply(lambda x: np.log(1+x))
asri

In [ ]:
# variance of asri
var_asri = asri.var()
var_asri

In [ ]:
# volatility
pwon_vol = np.sqrt(var_pwon * 250)
asri_vol = np.sqrt(var_asri * 250)

pwon_vol, asri_vol

In [ ]:
# Plot volatility of both stock
test.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x * np.sqrt(250)).plot(kind='bar')

## Covariance

In [ ]:
# Log of percentage
test1 = test.pct_change().apply(lambda x: np.log(1+x))
test1.head()

In [ ]:
test1["PWON.JK"].cov(test1["ASRI.JK"])


## Correlation

In [ ]:
test1["PWON.JK"].corr(test1["ASRI.JK"])

## Expected Return

In [ ]:
# calculate log of percentage
test2 = test.pct_change().apply(lambda x: np.log(1+x))
test2.head()

### Weights

In [ ]:
# Define weights allocation
w = [0.2, 0.8]
e_r_ind = test2.mean()  # expected return individual
e_r_ind

In [ ]:
# Total expected return
e_r = (e_r_ind * w).sum()
e_r

## Build Optimal Risky Portfolio

In [ ]:
# import data
df = data.DataReader(["PWON.JK", "ASRI.JK", "BBCA.JK", "BMRI.JK"], "yahoo", start_date, end_date)
df = pd.DataFrame(df)
df.head()

In [ ]:
# Close price
df = df["Close"]
df.head()

## Covariance and Correlation matrix

In [ ]:
cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

In [ ]:
corr_matrix = df.pct_change().apply(lambda x: np.log(1+x)).corr()
corr_matrix

## Portfolio Variance

In [ ]:
w = {"PWON.JK":0.1, "ASRI.JK":0.2, "BBCA.JK":0.5, "BMRI.JK":0.2}
port_var = cov_matrix.mul(w, axis=0).mul(w, axis=1).sum().sum()
port_var

## Portfolio expected return

In [ ]:
# Yearly return for individual companies
ind_er = df.resample("Y").last().pct_change().mean()
ind_er

In [ ]:
# Portfolio return
w = [0.1, 0.2, 0.5, 0.2]
port_er = (w * ind_er).sum()
port_er

### Plotting Efficient Frontier

In [ ]:
# Volatility is given by the annual standard deviation. We multiply by 250 because there are 250 trading days/year.
ann_sd = df.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))
ann_sd

In [ ]:
assets = pd.concat([ind_er, ann_sd], axis=1) # Creating a table for visualising returns and volatility of assets
assets.columns = ['Returns', 'Volatility']
assets

In [ ]:
p_ret = []          # Define an empty array for portfolio returns
p_vol = []          # Define an empty array for portfolio volatility
p_weights = []      # Define an empty array for asset weights

num_assets = len(df.columns)
num_portfolios = 100000000  # saldo Rp. 100 juta

for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
    p_vol.append(ann_sd)

data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(df.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]

portfolios = pd.DataFrame(data)
portfolios.head()